이 노트북은 [케라스 창시자에게 배우는 딥러닝 2판](https://tensorflow.blog/kerasdl2/)의 예제 코드를 담고 있습니다.

<table align="left">
    <tr>
        <td>
            <a href="https://colab.research.google.com/github/rickiepark/deep-learning-with-python-2nd/blob/main/chapter13_best-practices-for-the-real-world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
        </td>
    </tr>
</table>

# 실전 문제 해결을 위한 모범 사례

## 모델의 최대 성능을 끌어 내기

### 하이퍼파라미터 최적화

#### KerasTuner 사용하기

In [1]:
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 13.5 MB/s eta 0:00:00


**KerasTuner 모델 구축 함수**

In [2]:
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
    units = hp.Int(name="units", min_value=16, max_value=64, step=16)
    model = keras.Sequential([
        layers.Dense(units, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])
    return model

**KerasTuner의 `HyperModel`**

In [3]:
import kerastuner as kt

class SimpleMLP(kt.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        units = hp.Int(name="units", min_value=16, max_value=64, step=16)
        model = keras.Sequential([
            layers.Dense(units, activation="relu"),
            layers.Dense(self.num_classes, activation="softmax")
        ])
        optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])
        return model

hypermodel = SimpleMLP(num_classes=10)

Using TensorFlow backend


<ipython-input-3-643dcc05e516>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [4]:
tuner = kt.BayesianOptimization(
    build_model,
    objective="val_accuracy",
    # 코랩에서 정상 실행만 확인하기 위해 최대 탐색 횟수를 100에서 10으로 줄입니다
    max_trials=10, # 100
    executions_per_trial=2,
    directory="mnist_kt_test",
    overwrite=True,
)

In [5]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


In [6]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255
x_train_full = x_train[:]
y_train_full = y_train[:]
num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),
]
tuner.search(
    x_train, y_train,
    batch_size=128,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=2,
)

Trial 10 Complete [00h 01m 14s]
val_accuracy: 0.9758500158786774

Best val_accuracy So Far: 0.9758500158786774
Total elapsed time: 00h 12m 40s


**최상의 하이퍼파라미터 설정 확인하기**

In [7]:
top_n = 4
best_hps = tuner.get_best_hyperparameters(top_n)

In [8]:
def get_best_epoch(hp):
    model = build_model(hp)
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor="val_loss", mode="min", patience=10)
    ]
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=100,
        batch_size=128,
        callbacks=callbacks)
    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}")
    return best_epoch

In [9]:
best_hps

In [10]:
best_hps[0].Choice(name="optimizer", values=["rmsprop", "adam"])

'rmsprop'

In [11]:
model = build_model(best_hps[0])

In [12]:
from keras_tuner.engine.hyperparameters import Int

In [13]:
best_hps[0]._conditions_are_active(Int(name="units", min_value=0, max_value=1).conditions)

True

In [14]:
best_hps[0]._hps

defaultdict(list,
            {'units': [Int(name: 'units', min_value: 16, max_value: 64, step: 16, sampling: linear, default: 16)],
             'optimizer': [Choice(name: 'optimizer', values: ['rmsprop', 'adam'], ordered: False, default: rmsprop)]})

In [15]:
best_hps[0]._exists('units', [])

True

In [16]:
best_hps[0].values

{'units': 64, 'optimizer': 'rmsprop'}

In [17]:
best_hps[0].Int(name="units", min_value=0, max_value=1)

64

In [18]:
model.get_config()

{'name': 'sequential_1',
 'layers': [{'module': 'keras.layers',
   'class_name': 'Dense',
   'config': {'name': 'dense_2',
    'trainable': True,
    'dtype': 'float32',
    'units': 64,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'module': 'keras.initializers',
     'class_name': 'GlorotUniform',
     'config': {'seed': None},
     'registered_name': None},
    'bias_initializer': {'module': 'keras.initializers',
     'class_name': 'Zeros',
     'config': {},
     'registered_name': None},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None},
   'registered_name': None},
  {'module': 'keras.layers',
   'class_name': 'Dense',
   'config': {'name': 'dense_3',
    'trainable': True,
    'dtype': 'float32',
    'units': 10,
    'activation': 'softmax',
    'use_bias': True,
    'kernel_initializer': {'module': 'keras.initializers',
     'class_name': 'Gloro

In [19]:
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    model.fit(
        x_train_full, y_train_full,
        batch_size=128, epochs=int(best_epoch * 1.2))
    return model

best_models = []
for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models.append(model)

Epoch 1/100
391/391 [==============================] - 2s 4ms/step - loss: 0.4213 - accuracy: 0.8859 - val_loss: 0.2411 - val_accuracy: 0.9318
Epoch 2/100
391/391 [==============================] - 1s 4ms/step - loss: 0.2200 - accuracy: 0.9371 - val_loss: 0.1795 - val_accuracy: 0.9511
Epoch 3/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1696 - accuracy: 0.9507 - val_loss: 0.1560 - val_accuracy: 0.9587
Epoch 4/100
391/391 [==============================] - 2s 4ms/step - loss: 0.1383 - accuracy: 0.9597 - val_loss: 0.1334 - val_accuracy: 0.9626
Epoch 5/100
391/391 [==============================] - 2s 4ms/step - loss: 0.1173 - accuracy: 0.9660 - val_loss: 0.1262 - val_accuracy: 0.9654
Epoch 6/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1017 - accuracy: 0.9710 - val_loss: 0.1200 - val_accuracy: 0.9661
Epoch 7/100
391/391 [==============================] - 1s 3ms/step - loss: 0.0892 - accuracy: 0.9745 - val_loss: 0.1160 - val_accuracy: 0.9676

In [20]:
best_models = tuner.get_best_models(top_n)

#### 올바른 검색 공간을 만드는 기술

#### 하이퍼파라미터 튜닝의 미래: 자동화된 머신 러닝

### 모델 앙상블

## 대규모 모델 훈련하기

### 혼합 정밀도로 GPU에서 훈련 속도 높이기

#### 부동 소수점 정밀도 이해하기

In [21]:
import tensorflow as tf
import numpy as np
np_array = np.zeros((2, 2))
tf_tensor = tf.convert_to_tensor(np_array)
tf_tensor.dtype

tf.float64

In [22]:
np_array = np.zeros((2, 2))
tf_tensor = tf.convert_to_tensor(np_array, dtype="float32")
tf_tensor.dtype

tf.float32

#### 혼합 정밀도로 훈련하기

In [23]:
from tensorflow import keras
keras.mixed_precision.set_global_policy("mixed_float16")

### 다중 GPU 훈련

#### 두 개 이상의 GPU 활용하기

#### 단일 호스트, 다중 장치 동기 훈련

### TPU 훈련

#### 구글 코랩에서 TPU 사용하기

#### 스텝 융합을 활용하여 TPU 활용도 높이기

## 요약